In [ ]:
import pandas as pd 
from BorutaShap import BorutaShap
from sklearn.ensemble import RandomForestClassifier

# Include Tentative Boruta Shap 

In [ ]:
def BorutaShap_FS (X, y,method_option) :
    #modelshap = RandomForestClassifier(n_jobs=-1,n_estimators=100, class_weight='balanced_subsample', max_depth=5, random_state=100)
    modelshap = RandomForestClassifier(n_jobs=-1,n_estimators=100, max_depth=5, random_state=100)

    # define model for resp. classifier
    modelshap.fit(X,y)
    feature_names = np.array(X.columns)
    # define Boruta Sahp feature selection method
    feature_selector = BorutaShap(model=modelshap,
                              importance_measure=method_option,
                              classification=True)  # find all relevant features
    feature_selector.fit(X,y,n_trials=100,sample = False, verbose = True,random_state=100)  
    #feature_selector.plot(which_features='accepted',figsize=(20,10))
    tentative=X.loc[:,feature_selector.tentative]
    selected=feature_selector.Subset()
    selten=pd.concat([selected,tentative],axis=1)
    # call transform() on X to filter it down to selected features
    return  selten

In [ ]:
list_method_t=['shap','gini']
list_borshap_t = []
#order=0

for m in list_method_t:
    Output_t = BorutaShap_FS(retransformed_train,y_train,m)
    list_borshap_t.append(Output_t)
    
for r in range(len(list_borshap_t)):
    print(list_borshap_t[r])

### return unscaled DF RFE

In [ ]:
def RFE_FS (X, y,classify) :
    scaler = MinMaxScaler()
    X_scaled= pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
    feature_names = np.array(X_scaled.columns)
    if classify == 'RF':
    # define random forest classifier
        model = RandomForestClassifier(n_jobs=-1, class_weight='balanced_subsample', max_depth=5, random_state=100)
       
    if classify== 'SVM':
        model = SVC(kernel='linear',C=5)
        #rfe = RFECV(estimator = model,scoring='accuracy')
    # find all relevant features
    model.fit(X_scaled, y)
    rfe = RFE(estimator = model,n_features_to_select = 15)
    rfe.fit(X_scaled,y)

     # zip feature names, ranks, and decisions 
    feature_ranks = list(zip(feature_names, 
                             rfe.ranking_, 
                             rfe.support_))

    final_features_rfe = list()
    indexes = np.where(rfe.ranking_ <= 2)
    for x in np.nditer(indexes):
        final_features_rfe.append(feature_names[x])
    
    
    # unscale the data before return
    X_unscaled=pd.DataFrame(scaler.inverse_transform(X_scaled), columns=X_scaled.columns)
    ff_rfe=pd.DataFrame(X_unscaled.filter(final_features_rfe))
    

 # call transform() on X to filter it down to selected features
    return  ff_rfe

In [ ]:
list_clf=['RF','SVM']
list_rfe = []
#list_ff_rfe=[]

for c in list_clf:
    #bf_rfe, ff_rfe = RFE_FS(imputed_train,y_train,c)
    bf_rfe = RFE_FS(retransformed_train,y_train,c)
    list_rfe.append(bf_rfe)
    #list_ff_rfe.append (ff_rfe)
    
for l in range(len(list_rfe)):
    print("list of dataframes \n\n",list_rfe[l])
    #print("list of chosen features \n\n", list_ff_rfe[l])